# Trabalho de AEDV (A1)
   
#### Dupla dinâmica: $(igor)^2 = -(gor)^2$
    Igor Cortes Junqueira
    Igor Patricio Michels

## Escolha da base de dados

O primeiro passo do trabalho foi a escolha da base de dados. Para tanto optamos por otimizar o tempo fazendo a escolha de uma base que pode nos porporcionar a escolha da próxima série a ser assistida. Dessa forma, escolhemos uma base do Kaggle com séries em diversas plataformas, a qual pode ser encontrada [aqui](https://www.kaggle.com/ruchi798/tv-shows-on-netflix-prime-video-hulu-and-disney).

Um problema dessa base é que ela não é muito informativa, isto é, uma das informações que consideramos de maior relevância, o gênero da série, não está presente nessa base. Dessa forma, buscamos novas bases de dados no Kaggle, uma da [Netflix](https://www.kaggle.com/shivamb/netflix-shows), uma da [Amazon Prime](https://www.kaggle.com/nilimajauhari/amazon-prime-tv-shows) e uma da [Disney Plus](https://www.kaggle.com/unanimad/disney-plus-shows), as quais possuem a indicação do gênero.

Para utilizar as 4 bases descritas, realizamos um merge entre elas a partir do título, incluindo o gênero da série na nossa base. As demais informações presentes nas últimas 3 bases foram descartadas, uma vez que nem todas tinham aquela informação. Da base original descartamos todos os títulos que estavam presentes apenas na plataforma Hulu, uma vez que esta plataforma não está disponível no Brasil.

Abaixo temos a preparação do ambiente, com a importação das bibliotecas utilizadas, bem como a importação das bancos de dados.

## Importação e limpeza dos dados

In [1]:
library(readr)
library(stringr)
library(rlist)
library(dplyr)
library(ggplot2)
shows <- read_csv("tv_shows.csv")
disney <- read_csv("disney_plus_shows.csv")
netflix <- read_csv("netflix_titles.csv")
prime <- read_csv("Prime TV Shows Data set.csv")

Warning message:
"package 'rlist' was built under R version 3.6.3"Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'ggplot2' was built under R version 3.6.3"Parsed with column specification:
cols(
  ID = col_character(),
  Title = col_character(),
  Year = col_double(),
  Age = col_character(),
  IMDb = col_double(),
  Rotten_Tomatoes = col_character(),
  Netflix = col_double(),
  Hulu = col_double(),
  Prime_Video = col_double(),
  DisneyPlus = col_double(),
  type = col_double()
)
Warning message:
"85 parsing failures.
row col   expected    actual           file
 63  -- 11 columns 1 columns 'tv_shows.csv'
 69  -- 11 columns 1 columns 'tv_shows.csv'
423  -- 11 columns 1 columns 'tv_shows.csv'
453  -- 11 columns 1 columns 'tv_shows.csv'
528  -- 11 c

Feito isso, iremos compor a base do trabalho, isto é, fazer o merge dos bancos utilizados, bem como a exclusão das séries exclusivas da Hulu.

In [2]:
# Limpando o databse conjunto para os dados relevantes

cshows <- shows %>% mutate(Streamings = paste(sep = " ", ifelse(Netflix == 1, "Netflix", ""),
                                                         ifelse(Prime_Video == 1, "Prime", ""),
                                                         ifelse(DisneyPlus == 1, "Disney", "")) %>%
                                        trimws() %>%
                                        str_replace(" ",", ")) %>%
                                        select(-Netflix, -Hulu, -Prime_Video, -DisneyPlus, -type, -ID)

cshows <- subset(cshows, Streamings != "NA, NA NA" & Streamings != "")
cshows <- cshows %>% mutate(Rotten_Tomatoes = as.numeric(sub("%", "", Rotten_Tomatoes)))

# Importanto Genero dos demais databases

t1 <- netflix %>% mutate(Title = title, Genre = listed_in) %>% select(Title, Genre) 
t2 <- prime %>% mutate(Title = ) %>% select(Title, Genre)
t3 <- disney %>% mutate(Title = title, Genre = genre) %>% select(Title, Genre)
gb <- c("Title", "Genre")
t <- full_join(t1, full_join(t2, t3, by = gb), by = gb)

cshows <- left_join(cshows, t, by = 'Title')
glimpse(cshows)

Rows: 4,143
Columns: 7
$ Title           <chr> "Breaking Bad", "Stranger Things", "Money Heist", "She~
$ Year            <dbl> 2008, 2016, 2017, 2010, 2015, 2005, 2011, 2005, 2005, ~
$ Age             <chr> "18+", "16+", "18+", "16+", "18+", "16+", "18+", "16+"~
$ IMDb            <dbl> 9.5, 8.8, 8.4, 9.1, 8.7, 8.9, 8.8, 8.4, 8.4, 8.8, 9.2,~
$ Rotten_Tomatoes <dbl> 96, 93, 91, 78, 97, 81, 83, 93, 93, 92, 100, 81, 94, 8~
$ Streamings      <chr> "Netflix", "Netflix", "Netflix", "Netflix", "Netflix",~
$ Genre           <chr> "Crime TV Shows, TV Dramas, TV Thrillers", "TV Horror,~


Como utilizamos 3 bases distintas para incluir o gênero da série, é natural que o gênero esteja totalmente despadronizado, dessa forma estamos criando uma lista de gêneros para facilitar nossas queries.

In [ ]:
x <- ""
for(g in cshows[,'Genre'][[1]]){
    c <- str_replace_all(g,"TV","") %>%
         str_replace_all("Reality-", "Reality-Show") %>%
         str_replace_all("Sci-Fi", "Sci-fi") %>%
         str_replace_all("Sci-fi comedy", "") %>%
         str_replace_all(" &", ",") %>%
         str_replace_all("Movies", "") %>%
         str_replace_all("Shows", "") %>%
         str_replace_all("/", "") %>%
         str_replace_all("Mysteries", "Mystery") %>%
         str_replace_all("Comedies", "Comedy") %>%
         str_replace_all("Documentaries", "Documentary") %>%
         str_replace_all("Dramas", "Drama") %>%
         str_replace_all("Musicals", "Musical") %>%
         str_replace_all("Thrillers", "Thriller") %>%
         str_replace_all("Talk-Show", "") %>%
         str_replace_all("Romantic", "Romance") %>%
         str_replace_all("Anime Series", "Anime") %>%
         str_replace_all("Kids\'", "") %>%
         str_replace_all("Kids\'", "")

    x <- paste(x, trimws(c), sep = ",")
}

gens <- strsplit(x, ",")[[1]] %>%
        trimws() %>%
        unique()

gens <- gens[gens != ""]
gens

## Análise Exploratória

In [ ]:
varpop <- function(x){
  n <- length(x)
  var(x)*(n-1)/n
}

In [ ]:
plataforma <- table(cshows$Streamings)
plataforma

In [ ]:
barplot(sort(plataforma))

In [ ]:
plataforma <- prop.table(plataforma)
plataforma

In [ ]:
barplot(sort(plataforma))

Note que mais que 50% das séries disponíveis se encontram no Amazon Prime (48.85% da Amazon Prime e 2.24% que estão na Netflix e na Amazon Prime). Outro fato interessante é que o Disney Plus tem menos que 5% das séries e, além disso, nenhuma série se encontra no Disney Plus e na Amazon Prime simultaneamente.

Um outro dado de nosso interesse é a avaliação da crítica, dessa forma, analisando as notas, quanto ao IMDb, temos.

In [ ]:
quantile(cshows[, "IMDb"], na.rm = TRUE)

In [ ]:
stripchart(cshows[, "IMDb"], xlab = "Nota IMDb")

In [ ]:
cshows %>% 
ggplot(aes(x = IMDb)) +
geom_boxplot() +
labs(x = "Nota IMDb")

Já de acordo com o Rotten Tomatoes

In [ ]:
quantile(cshows[, "Rotten_Tomatoes"], na.rm = TRUE)

In [ ]:
stripchart(cshows[, "Rotten_Tomatoes"], xlab = "Nota Rotten Tomatoes (%)")

In [ ]:
cshows %>% 
ggplot(aes(x = Rotten_Tomatoes)) +
geom_boxplot() +
labs(x = "Nota Rotten Tomatoes (%)")

Também é possível fazer algumas análises quanto as notas de acordo com a série, dando ênfase na plataforma de origem. Nos gráficos abaixo estão as melhores séries de 2020, por exemplo.

In [ ]:
# Melhores séries de 2020 pelo IMDb (nota > 8)
cshows %>% filter(Year==2020, IMDb>8) %>% arrange(-IMDb) %>%
    ggplot(aes(reorder(Title,IMDb),IMDb)) + geom_col(aes(fill=Streamings)) +
    coord_flip(ylim=c(8,9.3)) + xlab("Séries de 2020") + ylab("Nota IMDb") +
    geom_text(aes(label = IMDb), hjust = -.5)

In [ ]:
# Melhores séries de 2020 pelo Rotten Tomatoes (nota > 80)
cshows %>% filter(Year==2020, Rotten_Tomatoes>=80) %>%
    ggplot(aes(reorder(Title,Rotten_Tomatoes),Rotten_Tomatoes)) + geom_col(aes(fill=Streamings)) +
    coord_flip(ylim=c(75,105)) + xlab("Séries de 2020") + ylab("Nota Rotten Tomatoes") +
    geom_text(aes(label = Rotten_Tomatoes), hjust = -.5)

Outra análise interessante se dá pela faixa etária da série. Dessa forma, é possível também analisar a quantidade de títulos por faixa etária e plataforma

In [ ]:
plataforma <- table(cshows$Streamings, cshows$Age)
plataforma

In [ ]:
barplot(plataforma,
        beside = TRUE,
        legend.text = c("Disney", "Netflix", "Netflix, Disney", "Netflix, Prime", "Prime"),
        col = c("#f8766d", "#a3a500", "#00bf7d", "#00b0f6", "#e76bf3"),
        ylim = c(0, 500))

Uma outra visualização dos mesmos dados, mas com a inclusão de uma série temporal, se dá conforme abaixo

In [ ]:
# Séries por ano, por faixa etária. Tirei idades 13+ (poucas séries) NA (não informativo)
cshows %>% 
filter(Age != "13+", Age != "NA") %>% 
    ggplot(aes(x = Year, fill = Streamings)) +
    geom_histogram(binwidth = 1, boundary = 0) +
    facet_wrap(~ Age, ncol = 2)+ 
    labs(x = "Ano",
    y = "Quantidade de séries",
    title = "Histograma de séries por ano",
    subtitle = "Separação por classificação etária")

Agora, uma pergunta natural é: "As séries de uma plataforma são melhores que de outra?", bem como a pergunta "Há alguma correlação entre a plataforma de uma série e sua nota?".

Podemos ver a correlação entre plataforma e nota abaixo. Primeiramente pelo IMDb.

In [ ]:
notas_disney <- cshows %>% filter(Streamings == "Disney", !is.na(IMDb))
var_disney <- varpop(notas_disney$IMDb)
n_disney <- length(notas_disney$IMDb)

notas_netflix <- cshows %>% filter(Streamings == "Netflix", !is.na(IMDb))
var_netflix <- varpop(notas_netflix$IMDb)
n_netflix <- length(notas_netflix$IMDb)

notas_prime <- cshows %>% filter(Streamings == "Prime", !is.na(IMDb))
var_prime <- varpop(notas_prime$IMDb)
n_prime <- length(notas_prime$IMDb)

notas_netflix_disney <- cshows %>% filter(Streamings == "Netflix,  Disney", !is.na(IMDb))
var_netflix_disney <- varpop(notas_netflix_disney$IMDb)
n_netflix_disney <- length(notas_netflix_disney$IMDb)

notas_netflix_prime <- cshows %>% filter(Streamings == "Netflix, Prime", !is.na(IMDb))
var_netflix_prime <- varpop(notas_netflix_prime$IMDb)
n_netflix_prime <- length(notas_netflix_prime$IMDb)

notas <- cshows %>% filter(!is.na(IMDb))
var <- varpop(notas$IMDb)
n_IMDb <- n_disney + n_netflix + n_prime + n_netflix_disney + n_netflix_prime

var_barra <- (n_disney * var_disney + n_netflix * var_netflix + n_prime * var_prime + n_netflix_disney * var_netflix_disney + n_netflix_prime * var_netflix_prime)/n_IMDb
r² <- 1 - var_barra/var
r²

In [ ]:
X_squared <- chisq.test(cshows$Streamings, cshows$IMDb)
X_squared[1]

In [ ]:
C = sqrt(unname(X_squared[["statistic"]])/(length(cshows$Streamings) + unname(X_squared[["statistic"]])))
C

De onde percebe-se que aproximadamente 27% da nota do IMDb pode ser explicada através da Streaming.

Podemos ver o mesmo de acordo com o Rotten Tomatoes.

In [ ]:
notas_disney <- cshows %>% filter(Streamings == "Disney", !is.na(Rotten_Tomatoes))
var_disney <- varpop(notas_disney$Rotten_Tomatoes)
n_disney <- length(notas_disney$Rotten_Tomatoes)

notas_netflix <- cshows %>% filter(Streamings == "Netflix", !is.na(Rotten_Tomatoes))
var_netflix <- varpop(notas_netflix$Rotten_Tomatoes)
n_netflix <- length(notas_netflix$Rotten_Tomatoes)

notas_prime <- cshows %>% filter(Streamings == "Prime", !is.na(Rotten_Tomatoes))
var_prime <- varpop(notas_prime$Rotten_Tomatoes)
n_prime <- length(notas_prime$Rotten_Tomatoes)

notas_netflix_prime <- cshows %>% filter(Streamings == "Netflix, Prime", !is.na(Rotten_Tomatoes))
var_netflix_prime <- varpop(notas_netflix_prime$Rotten_Tomatoes)
n_netflix_prime <- length(notas_netflix_prime$Rotten_Tomatoes)

notas <- cshows %>% filter(!is.na(Rotten_Tomatoes))
var <- varpop(notas$Rotten_Tomatoes)
n_Rotten <- n_disney + n_netflix + n_prime + n_netflix_disney + n_netflix_prime

var_barra <- (n_disney * var_disney + n_netflix * var_netflix + n_prime * var_prime + n_netflix_prime * var_netflix_prime)/n_Rotten
r² <- 1 - var_barra/var
r²

In [ ]:
X_squared <- chisq.test(cshows$Streamings, cshows$Rotten_Tomatoes)
X_squared[1]

In [ ]:
C = sqrt(unname(X_squared[["statistic"]])/(length(cshows$Streamings) + unname(X_squared[["statistic"]])))
C

De onde percebe-se que aproximadamente 23% da nota do Rotten Tomatoes pode ser explicada através da Streaming. Aqui vale ressaltar que a quantidade de séries observadas é bem menor, conforme pode-se observar abaixo

In [ ]:
print(n_IMDb) # Nota IMDb
print(n_Rotten) # Nota Rotten Tomatoes

Com essas porcentagens na explicação, podemos ver que a plataforma de Streaming não é de suma importância para o sucesso de uma série, embora tenha boa relevância.

Agora, vamos a análise das notas das séries de cada plataforma. Um detalhe interessante para isso é analisar a distribuição das notas por plataforma, assim

In [ ]:
stripchart(cshows$IMDb ~ cshows$Streamings, xlab = "Nota IMDb", ylab = "Plataforma")

In [ ]:
stripchart(cshows$Rotten_Tomatoes ~ cshows$Streamings, xlab = "Nota Rotten Tomatoes", ylab = "Plataforma")

Em virtude da quantidade de séries em plataformas como Netflix e Amazon Prime, onde temos várias séries, muitas partilhando a mesma nota, uma melhor noção de como elas se comportam se dá pelo boxplot dos dados

In [ ]:
cshows %>% 
ggplot(aes(x = Streamings, y = IMDb)) +
geom_boxplot() +
labs(x = "Plataforma",
y = "Nota IMDb",
title = "Box plot das Notas IMDb por plataforma")

In [ ]:
cshows %>% filter(Streamings != "Netflix,  Disney") %>% 
ggplot(aes(x = Streamings, y = Rotten_Tomatoes)) +
geom_boxplot() +
labs(x = "Plataforma",
y = "Nota IMDb",
title = "Box plot das Notas Rotten Tomatoes por plataforma")

Podemos também ver como as notas das séries se comportam ao comparar as duas notas obtidas.

In [ ]:
cshows %>% filter(Streamings != "Netflix,  Disney") %>% 
ggplot(aes(x = IMDb, y = Rotten_Tomatoes, color = Streamings)) +
labs(x = "IMDb", y = "Rotten Tomatoes", title = "Nota IMDb x Rotten Tomatoes") + 
geom_point()

Note que a análise da quantidade de séries com cada nota fica complicada nesse gráfico, dessa forma, podemos dividir as plataformas em subgráficos:

In [ ]:
cshows %>% filter(Streamings != "Netflix,  Disney", !is.na(IMDb), !is.na(Rotten_Tomatoes)) %>% 
ggplot(aes(x = IMDb, y = Rotten_Tomatoes, color = Streamings)) +
labs(x = "IMDb", y = "Rotten Tomatoes", title = "Nota IMDb x Rotten Tomatoes") + 
geom_point() +
facet_wrap(~ Streamings, ncol = 2)

In [ ]:
glimpse(cshows)

Por meio do Gráfico de Simetria entre IMDb e Rotten Tomatoes podemos comparar as críticas das séries.

In [ ]:
notas <- cshows %>% filter(!is.na(IMDb), !is.na(Rotten_Tomatoes)) %>% mutate(IMDb = 10*IMDb)

# x <- median(notas$IMDb) - notas$IMDb
# y <- notas$Rotten_Tomatoes - median(notas$Rotten_Tomatoes)

x <- notas$IMDb
y <- notas$Rotten_Tomatoes
plot(sort(x), sort(y), pch = 19, xlab = "IMDb", ylab = "Rotten Tomatoes", col = "darkblue", 
     xlim = c(0, max(x)), ylim = c(0, max(y)))
abline(0, 1)

Perceba que, na maioria das séries, as críticas divergem de opinião. Note também que o IMDb não credita muitos valores extremos, isto é, não temos nenhuma nota abaixo de 20 e poucas acima de 90, enquanto o Rotten Tomatoes essas quantidades são maiores.

In [ ]:
## qq plot
cshows %>% ggplot(aes(sample=IMDb))+stat_qq()

In [ ]:
## qq plot
cshows %>% ggplot(aes(sample=Rotten_Tomatoes))+stat_qq()

## Considerações finais

Para a elaboração do trabalho nossa dupla optou por se reunir em uma chamada via zoom e realizar a montagem do banco a ser utilizado. Após isso foi realizado de forma mais individual, com cada membro criando algumas visualizações e depois juntando ambas.